In [1]:
import os
GPU_id = 4
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_id)

In [2]:
import cudf
import pandas as pd
import numpy as np
import xgboost as xgb
import os
import time
import nvstrings
from librmm_cffi import librmm
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, wait
%matplotlib inline

In [3]:
%%time
path = '/datasets/trivago/data/'
cu_df = pd.read_csv(path + 'item_metadata.csv' )
cu_df = cudf.DataFrame.from_pandas(cu_df)
# res = orchestrate_bin(path, cats_col)

CPU times: user 3.09 s, sys: 1.29 s, total: 4.38 s
Wall time: 4.95 s


In [4]:
import nvstrings, nvcategory

In [5]:
%%time
props_nv = nvstrings.from_csv(path + 'item_metadata.csv', 1)

CPU times: user 276 ms, sys: 248 ms, total: 524 ms
Wall time: 570 ms


In [6]:
%%time
new_spl = props_nv.split(delimiter='|')

CPU times: user 168 ms, sys: 472 ms, total: 640 ms
Wall time: 1.56 s


In [7]:
%%time
cats_nv = nvcategory.from_strings_list(new_spl)

CPU times: user 244 ms, sys: 272 ms, total: 516 ms
Wall time: 1.06 s


In [8]:
%%time
cats_nv = cats_nv.keys().to_host()

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 43.5 ms


In [9]:
cats_nv.remove(None)

In [10]:
len(cats_nv)

157

In [11]:
%%time
# remove index 0 == None
for cat in cats_nv:
    new_col = props_nv.contains(cat)
    new_col = cudf.Series(new_col).astype(int)
    cu_df.add_column(f'is_{cat}', new_col)

CPU times: user 5.91 s, sys: 1.35 s, total: 7.26 s
Wall time: 9.25 s


In [12]:
%%time
cu_df.drop_column('properties')

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 3.34 ms


In [ ]:
cu_df.to_pandas().head(5)